<a href="https://colab.research.google.com/github/DaasDaham/Deep-Learning-NLP/blob/main/TextCNNImplementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
text_corpus = []

In [ ]:
import pandas as pd
import numpy as np
import torch
from torchtext.legacy import data

SEED = 1234
import pandas as pd
import numpy as np
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchtext

import nltk

import random
from sklearn.metrics import classification_report
%matplotlib inline  

In [ ]:
import chardet
chardet.detect(open('/content/Trec_Train_dataset.txt', 'rb').read())['encoding']

'ISO-8859-1'

In [ ]:
file1 = open('/content/Trec_Train_dataset.txt', 'r',encoding="ISO-8859-1")
Lines = file1.readlines()
for line in Lines:
    text_corpus.append(line.strip())

In [ ]:
import re
labels = []
for t in range(len(text_corpus)):
    labels.append(text_corpus[t][:text_corpus[t].index(":")])
    text_corpus[t] = text_corpus[t][text_corpus[t].index(" ")+1:]
    # text_corpus[t] = text_corpus[t].replace("'s", "")
    # text_corpus[t] = text_corpus[t].replace("'", "")
    # text_corpus[t] = text_corpus[t].replace("`", "")
    # text_corpus[t] = text_corpus[t].replace(",", "")
    # text_corpus[t] = text_corpus[t].replace("!", "")
    # text_corpus[t] = text_corpus[t].replace(".", "")
    # text_corpus[t] = text_corpus[t].replace(":", "")
    # text_corpus[t] = text_corpus[t].replace("?", "")
    # text_corpus[t] = re.sub('[0-9]+s', '', text_corpus[t])
    # text_corpus[t] = re.sub('[0-9]', '', text_corpus[t])
    # text_corpus[t] = re.sub(' +', ' ', text_corpus[t])
    text_corpus[t] = text_corpus[t].strip() 
text_corpus = pd.Series(text_corpus, name="Text")
labels = pd.Series(labels, name="Labels")

In [ ]:
df = pd.concat([text_corpus, labels], axis=1)
df

,Text,Labels
0,How did serfdom develop in and then leave Russ...,DESC
1,What films featured the character Popeye Doyle ?,ENTY
2,How can I find a list of celebrities ' real na...,DESC
3,What fowl grabs the spotlight after the Chines...,ENTY
4,What is the full form of .com ?,ABBR
...,...,...
5447,What 's the shape of a camel 's spine ?,ENTY
5448,What type of currency is used in China ?,ENTY
5449,What is the temperature today ?,NUM
5450,What is the temperature for cooking ?,NUM


In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['Labels'] = labelencoder.fit_transform(df['Labels'])
df


,Text,Labels
0,How did serfdom develop in and then leave Russ...,1
1,What films featured the character Popeye Doyle ?,2
2,How can I find a list of celebrities ' real na...,1
3,What fowl grabs the spotlight after the Chines...,2
4,What is the full form of .com ?,0
...,...,...
5447,What 's the shape of a camel 's spine ?,2
5448,What type of currency is used in China ?,2
5449,What is the temperature today ?,5
5450,What is the temperature for cooking ?,5


In [ ]:
maxl = 0
for i in df.iloc[:,0]:
    if(len(i.split(" "))>maxl):
        maxl = len(i.split(" "))
print(maxl, "<- Max length of sentence required for padding")

37 <- Max length of sentence required for padding


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, shuffle=True)
train

,Text,Labels
4974,Who wrote the bestselling Missionary Travels a...,3
428,What is Jane Goodall known for ?,1
1800,What comedian was banned from the Ed Sullivan ...,3
3828,What director portrayed the commandant of the ...,3
4874,"What common plant has a button , cap , cup , g...",2
...,...,...
4533,When did the last Americans leave Vietnam ?,5
1723,Where is Santa Lucia ?,4
736,What facial feature typically contains about 5...,1
5442,"What is the meaning of caliente , in English , ?",1


In [ ]:
train.to_csv("train.csv", index=False)
test.to_csv("test.csv", index=False)

In [ ]:
import spacy
spacy_en = spacy.load('en')
is_cuda = torch.cuda.is_available()
print("Cuda Status on system is {}".format(is_cuda))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Cuda Status on system is True


In [ ]:
TEXT = data.Field(sequential=True, tokenize="spacy", fix_length=38)
LABEL = data.LabelField(dtype=torch.long, sequential=False)

In [ ]:
train_data, _, test_data = data.TabularDataset.splits(
    path="/content/", train="train.csv", validation="test.csv", test="test.csv",format="csv", skip_header=True, 
    fields=[('Text', TEXT), ('Label', LABEL)]
)

In [ ]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 4361
Number of testing examples: 1091


In [ ]:
TEXT.build_vocab(train_data, vectors='glove.twitter.27B.50d', 
                 max_size=20000, min_freq=1)
LABEL.build_vocab(train_data)

In [ ]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 8175
Unique tokens in LABEL vocabulary: 6


In [ ]:
LABEL.vocab.freqs

Counter({'0': 74, '1': 927, '2': 992, '3': 995, '4': 659, '5': 714})

In [ ]:
BATCH_SIZE = 20

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# keep in mind the sort_key option 
train_iterator, _, test_iterator = data.BucketIterator.splits(
    (train_data, _, test_data), sort_key=lambda x: len(x.Text),
    batch_size=BATCH_SIZE,
    device=device)


In [ ]:
TEXT.vocab.vectors.shape

torch.Size([8175, 50])

In [ ]:
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, input_size):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1d_4 = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=2, kernel_size=4),
            nn.LeakyReLU(),
            nn.MaxPool1d(kernel_size=(input_size-4+1))
        )
        self.conv1d_3 = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=2, kernel_size=3),
            nn.LeakyReLU(),
            nn.MaxPool1d(kernel_size=(input_size-3+1))
        )
        self.conv1d_2 = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=2, kernel_size=2),
            nn.LeakyReLU(),
            nn.MaxPool1d(kernel_size=(input_size-2+1))
        )

    def forward(self, x):
        embedded = self.embedding(x)
        # embedded shape -> [sent_len, batch_size, emb_dim]
        embedded = embedded.permute(1,2,0)
        # embedded shape -> [batch_size, emb_dim, sent_len]
        out_conv_4 = self.conv1d_4(embedded)
        out_conv_3 = self.conv1d_3(embedded)
        out_conv_2 = self.conv1d_2(embedded)
        out_combined = torch.cat([out_conv_4,out_conv_3,out_conv_2],1)
        #print(out_conv_4.shape, out_conv_3.shape, out_combined.shape)
        return out_combined

In [ ]:
class TextCNN_Avg(nn.Module):
    def __init__(self, vocab_size, embedding_dim, input_size):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1d_4 = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=2, kernel_size=4),
            nn.LeakyReLU(),
            nn.AvgPool1d(kernel_size=(input_size-4+1))
        )
        self.conv1d_3 = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=2, kernel_size=3),
            nn.LeakyReLU(),
            nn.AvgPool1d(kernel_size=(input_size-3+1))
        )
        self.conv1d_2 = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=2, kernel_size=2),
            nn.LeakyReLU(),
            nn.AvgPool1d(kernel_size=(input_size-2+1))
        )

    def forward(self, x):
        embedded = self.embedding(x)
        # embedded shape -> [sent_len, batch_size, emb_dim]
        embedded = embedded.permute(1,2,0)
        # embedded shape -> [batch_size, emb_dim, sent_len]
        out_conv_4 = self.conv1d_4(embedded)
        out_conv_3 = self.conv1d_3(embedded)
        out_conv_2 = self.conv1d_2(embedded)
        out_combined = torch.cat([out_conv_4,out_conv_3,out_conv_2],1)
        #print(out_conv_4.shape, out_conv_3.shape)
        return out_combined

In [ ]:
class TextCNN_KMax(nn.Module):
    def __init__(self, vocab_size, embedding_dim, input_size, k):
        super().__init__()

        self.k = k

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv1d_4 = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=2, kernel_size=4),
            nn.LeakyReLU(),
        )
        self.conv1d_3 = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=2, kernel_size=3),
            nn.LeakyReLU(),
        )
        self.conv1d_2 = nn.Sequential(
            nn.Conv1d(in_channels=embedding_dim, out_channels=2, kernel_size=2),
            nn.LeakyReLU(),
        )
        self.fc = nn.Linear(6*self.k, 6)

    def kmax_pooling(self, x, dim, k):
        index = x.topk(k, dim = dim)[1].sort(dim = dim)[0]
        return x.gather(dim, index)

    def forward(self, x):
        embedded = self.embedding(x)
        # embedded shape -> [sent_len, batch_size, emb_dim]
        embedded = embedded.permute(1,2,0)
        # embedded shape -> [batch_size, emb_dim, sent_len]
        #print(embedded.shape)
        out_conv_4 = self.conv1d_4(embedded)
        out_k_4 = self.kmax_pooling(out_conv_4, 2, self.k)
        out_conv_3 = self.conv1d_3(embedded)
        out_k_3 = self.kmax_pooling(out_conv_3, 2, self.k)
        out_conv_2 = self.conv1d_2(embedded)
        out_k_2 = self.kmax_pooling(out_conv_2, 2, self.k)
        combined = torch.cat([out_k_4,out_k_3,out_k_2],1)
        combined = combined.view(-1,6*self.k)
        #print(combined.shape)
        combined = self.fc(combined)
        #print(combined.shape)
        out_combined = combined.view(-1,6,1)
        #print(out_conv_4.shape, out_conv_3.shape, out_k_4.shape, out_k_3.shape, combined.shape)
        return out_combined

In [ ]:
model = TextCNN_KMax(VOCAB_SIZE, EMB_DIM, INPUT_DIM, 5)
print(model)

TextCNN_KMax(
  (embedding): Embedding(8175, 50)
  (conv1d_4): Sequential(
    (0): Conv1d(50, 2, kernel_size=(4,), stride=(1,))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (conv1d_3): Sequential(
    (0): Conv1d(50, 2, kernel_size=(3,), stride=(1,))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (conv1d_2): Sequential(
    (0): Conv1d(50, 2, kernel_size=(2,), stride=(1,))
    (1): LeakyReLU(negative_slope=0.01)
  )
  (fc): Linear(in_features=30, out_features=6, bias=True)
)


In [ ]:
def predict_proba(preds):
    softmax = nn.Softmax(dim=1)
    preds= softmax(preds).squeeze(2)
    return preds

In [ ]:
def predict(preds):
    softmax = nn.Softmax(dim=1)
    preds, ind= torch.max(softmax(preds), 1)
    return ind

In [ ]:
def multi_class_accuracy(preds, y):
    y = y.view(len(y),1)
    #print(preds.shape, pred.shape, y.shape, "acc")
    softmax = nn.Softmax(dim=1)
    preds, ind= torch.max(softmax(preds), 1)
    correct = (ind == y).float()
    acc = correct.sum()/float(len(correct))
    #print(acc.item(), "ACCURACY")
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):    
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.Text)
        #print(predictions.shape, batch.Label.view(len(batch.Label),1).shape, batch.Label.view(len(batch.Label),1), "yoyo")
        loss = criterion(predictions, batch.Label.view(len(batch.Label),1))
        acc = multi_class_accuracy(predictions, batch.Label.view(len(batch.Label),1))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        #print((epoch_loss / len(iterator)),(epoch_acc / len(iterator)))
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [ ]:
def evaluate(model, iterator, criterion, make_roc_cm):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.Text)  
            loss = criterion(predictions, batch.Label.view(len(batch.Label),1))
            if make_roc_cm:
                probs = predict_proba(predictions) 
                pred_lbls = predict(predictions)
                for pred_lbl in pred_lbls.detach().cpu().numpy():
                    global_preds_x.append(list(pred_lbl)[0])
                for prob in probs.detach().cpu().numpy():
                    global_roc_x.append(list(prob))
                for lbl in batch.Label.view(len(batch.Label),1).detach().cpu().numpy():
                    global_roc_y.append(list(lbl)[0])
                
            acc = multi_class_accuracy(predictions, batch.Label.view(len(batch.Label),1))
            epoch_loss += loss.item()
            epoch_acc += acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
EMB_DIM = 50
INPUT_DIM = 38
VOCAB_SIZE = len(TEXT.vocab)
model = TextCNN_KMax(VOCAB_SIZE, EMB_DIM, INPUT_DIM, 5)
model.embedding.weight.data = TEXT.vocab.vectors.to(device)
model = model.to(device)
global_roc_x = []
global_roc_y = []
global_preds_x = []

In [ ]:
print(model)

TextCNN(
  (embedding): Embedding(8175, 50)
  (conv1d_4): Sequential(
    (0): Conv1d(50, 2, kernel_size=(4,), stride=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool1d(kernel_size=35, stride=35, padding=0, dilation=1, ceil_mode=False)
  )
  (conv1d_3): Sequential(
    (0): Conv1d(50, 2, kernel_size=(3,), stride=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool1d(kernel_size=36, stride=36, padding=0, dilation=1, ceil_mode=False)
  )
  (conv1d_2): Sequential(
    (0): Conv1d(50, 2, kernel_size=(2,), stride=(1,))
    (1): LeakyReLU(negative_slope=0.01)
    (2): MaxPool1d(kernel_size=37, stride=37, padding=0, dilation=1, ceil_mode=False)
  )
)


In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss()

In [ ]:
for ep in range(50):
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    test_loss, test_acc = evaluate(model, test_iterator, criterion, False)
    print("[Training Accuracy, Training Loss] ->", train_acc,train_loss,"[Test Accuracy, Test Loss] ->",test_acc, test_loss)

[Training Accuracy, Training Loss] -> 0.3938356239569786 1.5391814643389559 [Test Accuracy, Test Loss] -> 0.5359504201195456 1.3137830582532015
[Training Accuracy, Training Loss] -> 0.6673516081348402 1.0575605323325554 [Test Accuracy, Test Loss] -> 0.7139669580893083 0.8781905748627402
[Training Accuracy, Training Loss] -> 0.7737443049748739 0.7133768660837112 [Test Accuracy, Test Loss] -> 0.7823140642859719 0.6834989748217842
[Training Accuracy, Training Loss] -> 0.8271689619103523 0.5540992109334633 [Test Accuracy, Test Loss] -> 0.8023140614682978 0.6043212066997181
[Training Accuracy, Training Loss] -> 0.8666666819624704 0.45328444567377163 [Test Accuracy, Test Loss] -> 0.8168595162304965 0.557500567219474
[Training Accuracy, Training Loss] -> 0.8924657669785905 0.37896816907918346 [Test Accuracy, Test Loss] -> 0.8204958807338368 0.5304482809521935
[Training Accuracy, Training Loss] -> 0.9178082287039386 0.3109797736008962 [Test Accuracy, Test Loss] -> 0.8323140621185303 0.49355580

In [ ]:
print(global_roc_x, global_roc_y, global_preds_x)
print(global_roc_x.copy(), global_roc_y.copy(), global_preds_x.copy())


[] [] []
[] [] []


In [ ]:
test_loss, test_acc = evaluate(model, test_iterator, criterion, True)
print(test_loss, test_acc)

0.7006657879460941 0.8277686129916798


In [ ]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded_y = onehot_encoder.fit_transform(np.array(global_roc_y).reshape(-1,1))

In [ ]:
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, multilabel_confusion_matrix
from sklearn.metrics import f1_score
!matplotlib inline

def plot_roc(y_probs, y_test, plot_name):
    y_probs = np.array(y_probs)
    for i in range(6):
        lr_fpr, lr_tpr, _ = roc_curve(y_test[:,i],y_probs[:,i])
        plt.plot(lr_fpr, lr_tpr, marker='.', label='Number-'+str(i))
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    #plt.show()
    plt.savefig(plot_name+".png")
    plt.close()

def plot_confusion_matrix(y_pred, y_test, plot_name):
    y_test = np.array(y_test).reshape(-1,1)
    y_pred = np.array(y_pred).reshape(-1,1)
    cm = confusion_matrix(y_test, y_pred)
    print(cm)
    cmd = ConfusionMatrixDisplay(cm, display_labels=['0','1','2','3','4','5','6','7','8','9'])
    cmd.plot()
    #plt.show()
    plt.savefig(plot_name+".png")
    plt.close()

def get_f1_score(y_pred, y_test):
    y_test = np.array(y_test).reshape(-1,1)
    y_pred = np.array(y_pred).reshape(-1,1)
    print(f1_score(y_test, y_pred, average = None))

/bin/bash: matplotlib: command not found


In [ ]:
plot_roc(global_roc_x.copy(), onehot_encoded_y.copy(), "ROC_twitter50_Model")

In [ ]:
plot_confusion_matrix(global_preds_x.copy(), global_roc_y.copy(),"CF_twitter50_Model")

[[190  28   5   2   3   0]
 [ 23 179  44   4   8   0]
 [  6  19 197   4   6   3]
 [  1   4   5 171   1   0]
 [  2   9   2   3 160   0]
 [  0   0   4   1   0   7]]


In [ ]:
get_f1_score(global_preds_x.copy(), global_roc_y.copy())

[0.84444444 0.72032193 0.80081301 0.93188011 0.9039548  0.63636364]


In [ ]:
import dill
with open('best_model_3b.pkl', 'wb') as f:
    dill.dump(model, f)

# **FIN**
